In [ ]:
# Importing necessary modules
# NOTE: This code was built on top of the code from cheetah_PPO.py
 
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import gymnasium as gym
import imageio
import matplotlib.pyplot as plt
from collections import deque
from torch.distributions import MultivariateNormal
from tqdm import tqdm

class Actor(nn.Module):
    def __init__(self, state_dim, action_dim, action_std):
        """
        Initializes the Actor network with fixed action standard deviation.
        
        Args:
        - state_dim  : Dimension of the input state
        - action_dim : Dimension of the output action
        - action_std : Fixed standard deviation for the action distribution
        """
        super(Actor, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(state_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, action_dim),
        )
        # Fixed action std for exploration (could be made learnable if desired)
        self.action_std = action_std
    
    def forward(self, state):
        """
        Forward pass to compute action mean and covariance matrix.
        
        Args:
        - state: Input state tensor
        
        Returns:
        - action_mean : Mean of the action distribution
        - cov_matrix : Covariance matrix for exploration
        """
        action_mean = self.fc(state)
        cov_matrix = torch.diag(torch.full((action_mean.size(-1),), self.action_std**2)).unsqueeze(0)
        return action_mean, cov_matrix

class Critic(nn.Module):
    def __init__(self, state_dim):
        """
        Initializes the Critic network for value function approximation.
        
        Args:
        - state_dim: Dimension of the input state
        """
        super(Critic, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(state_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
        )

    def forward(self, state):
        """Returns the estimated value of the input state."""
        return self.fc(state)

class PPO:
    def __init__(self, state_dim, action_dim, action_std, lr, gamma, eps_clip, lam, epochs, num_mini_batches):
        """
        Enhanced PPO agent with GAE and minibatch updates.
        
        Args:
        - state_dim        : Dimension of the state space
        - action_dim       : Dimension of the action space
        - action_std       : Standard deviation for action exploration
        - lr               : Learning rate for both networks
        - gamma            : Discount factor for future rewards
        - eps_clip         : PPO clipping parameter
        - lam              : Lambda parameter for GAE
        - epochs           : Number of epochs for each PPO update
        - num_mini_batches : Number of minibatches for each update
        """
        self.actor  = Actor(state_dim, action_dim, action_std)
        self.critic = Critic(state_dim)
        
        self.actor_optimizer  = optim.Adam(self.actor.parameters(), lr=lr)
        self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=lr)
        
        self.gamma            = gamma
        self.eps_clip         = eps_clip
        self.action_std       = action_std
        self.lam              = lam  # Lambda parameter for GAE calculation
        self.epochs           = epochs
        self.num_mini_batches = num_mini_batches
    
    def select_action(self, state):
        """
        Selects an action using the current policy.
        
        Returns:
        - action         : Selected action
        - action_logprob : Log probability of the action
        - value          : Value estimate of the current state
        """
        with torch.no_grad():
            state_t = torch.FloatTensor(state).unsqueeze(0)
            action_mean, cov_matrix = self.actor(state_t)
            dist           = MultivariateNormal(action_mean, cov_matrix)
            action         = dist.sample()
            action_logprob = dist.log_prob(action)
            value          = self.critic(state_t).item()
            return action.squeeze(0).numpy(), action_logprob.item(), value
    
    def evaluate(self, states, actions):
        """
        Evaluates actions given states to compute probabilities and values.
        
        Returns:
        - action_logprobs: Log probabilities of the actions
        - state_values  : Value estimates of the states
        - dist_entropy  : Entropy of the action distribution
        """
        action_mean, cov_matrix = self.actor(states)
        dist            = MultivariateNormal(action_mean, cov_matrix)
        action_logprobs = dist.log_prob(actions)
        dist_entropy    = dist.entropy()
        state_values    = self.critic(states)
        return action_logprobs, state_values, dist_entropy

def compute_gae(rewards, values, dones, gamma, lam):
    """
    Computes Generalized Advantage Estimation (GAE).
    
    Args:
    - rewards : List of rewards for each timestep
    - values  : List of value estimates (length T+1, includes next state)
    - dones   : List of done flags
    - gamma   : Discount factor
    - lam     : GAE lambda parameter
    
    Returns:
    - advantages : GAE advantages
    - returns    : Computed returns (advantages + values)
    """
    advantages = np.zeros(len(rewards), dtype=np.float32)
    gae = 0
    for i in reversed(range(len(rewards))):
        # Delta = rt + (gamma)V(t+1) - V(t)
        delta = rewards[i] + gamma * values[i+1] * (1 - dones[i]) - values[i]
        # GAE(t) = (delta)t + ((gamma.(lambda))(delta)t+1) + ((gamma.λ)^2(delta)t+2 + ...
        gae = delta + gamma * lam * (1 - dones[i]) * gae
        advantages[i] = gae
    returns = advantages + values[:-1]
    return advantages, returns

def collect_rollout(env, agent, timesteps):
    """
    Collects experience from the environment.
    Enhanced version that tracks values for GAE computation.
    
    Returns:
    - states, actions, rewards, logprobs, dones : Experience data
    - values : Value estimates including the final state
    """
    states   = []
    actions  = []
    logprobs = []
    rewards  = []
    dones    = []
    values   = []

    state, _ = env.reset(seed=None)
    done = False

    # Get initial state value for GAE
    state_t = torch.FloatTensor(state).unsqueeze(0)
    value = agent.critic(state_t).item()

    for _ in range(timesteps):
        action, action_logprob, val = agent.select_action(state)
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated

        # Store transition data
        states.append(state)
        actions.append(action)
        logprobs.append(action_logprob)
        rewards.append(reward)
        dones.append(float(done))
        values.append(val)

        state = next_state
        if done:
            state, _ = env.reset(seed=None)

    # Get value of final state for GAE calculation
    state_t    = torch.FloatTensor(state).unsqueeze(0)
    next_value = agent.critic(state_t).item()
    values.append(next_value)

    return states, actions, rewards, logprobs, dones, values

def update(agent, states, actions, rewards, logprobs, dones, values, gamma, eps_clip):
    """
    Updates the policy and value networks using PPO with GAE and minibatch updates.
    
    Key differences from basic PPO:
    1. Uses GAE for advantage estimation
    2. Implements minibatch updates
    3. Multiple epochs per update
    """
    # Convert collected data to tensors
    states       = torch.FloatTensor(states)
    actions      = torch.FloatTensor(actions)
    old_logprobs = torch.FloatTensor(logprobs)

    # Compute advantages using GAE
    advantages, returns = compute_gae(rewards, values, dones, gamma, agent.lam)
    advantages          = torch.FloatTensor(advantages)
    returns             = torch.FloatTensor(returns)

    # Normalize advantages for training stability
    advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)

    # Minibatch training
    batch_size      = len(states)
    mini_batch_size = batch_size // agent.num_mini_batches

    for _ in range(agent.epochs):
        # Shuffle data for each epoch
        indices = np.arange(batch_size)
        np.random.shuffle(indices)
        
        # Update in minibatches
        for start in range(0, batch_size, mini_batch_size):
            end    = start + mini_batch_size
            mb_idx = indices[start:end]

            # Get minibatch data
            mb_states       = states[mb_idx]
            mb_actions      = actions[mb_idx]
            mb_returns      = returns[mb_idx]
            mb_advantages   = advantages[mb_idx]
            mb_old_logprobs = old_logprobs[mb_idx]

            # PPO update steps
            logprobs, state_values, dist_entropy = agent.evaluate(mb_states, mb_actions)
            ratios = torch.exp(logprobs - mb_old_logprobs)
            
            # PPO policy loss with clipping
            surr1       = ratios * mb_advantages
            surr2       = torch.clamp(ratios, 1 - eps_clip, 1 + eps_clip) * mb_advantages
            policy_loss = -torch.min(surr1, surr2).mean()

            # Value loss and entropy bonus
            value_loss = nn.MSELoss()(state_values.squeeze(), mb_returns)
            loss = policy_loss + 0.5 * value_loss - 0.01 * dist_entropy.mean()

            # Gradient update
            agent.actor_optimizer.zero_grad()
            agent.critic_optimizer.zero_grad()
            loss.backward()
            agent.actor_optimizer.step()
            agent.critic_optimizer.step()

def run_single_trial(trial_num, env_name):
    """
    Runs a single training trial.
    
    Args:
    - trial_num: Current trial number for tracking
    - env_name : Name of the gymnasium environment
    """
    print(f"\nStarting Trial {trial_num + 1}/5 with environment {env_name}")
    
    # Initialize environment and get dimensions
    env        = gym.make(env_name, render_mode="rgb_array")
    state_dim  = env.observation_space.shape[0]
    action_dim = env.action_space.shape[0]
    action_std = 0.5

    # Set hyperparameters
    lr       = 3e-4
    gamma    = 0.99
    eps_clip = 0.2
    lam      = 0.95          # Lambda for GAE
    epochs   = 10         # PPO update epochs
    num_mini_batches      = 32
    episodes              = 1000
    timesteps_per_rollout = 2048

    # Set seeds for reproducibility
    torch.manual_seed(trial_num)
    np.random.seed(trial_num)

    # Initialize PPO agent
    ppo = PPO(state_dim, action_dim, action_std, lr, gamma, eps_clip, lam, epochs, num_mini_batches)

    # Training tracking variables
    trial_rewards = []
    avg_rewards = deque(maxlen=100)

    # Main training loop
    for episode in tqdm(range(episodes), desc=f"Trial {trial_num + 1}"):
        # Collect experience and update policy
        states, actions, rewards, logprobs, dones, values = collect_rollout(env, ppo, timesteps_per_rollout)
        update(ppo, states, actions, rewards, logprobs, dones, values, gamma, eps_clip)

        # Track and log progress
        episode_reward = sum(rewards)
        trial_rewards.append(episode_reward)
        avg_rewards.append(episode_reward)

        if episode % 10 == 0:
            avg_reward = np.mean(list(avg_rewards))
            print(f"\nTrial {trial_num + 1}, Episode {episode}, Average Reward: {avg_reward:.2f}")

    # Record demonstration video
    try:
        frames    = []
        state, _  = env.reset()
        done      = False
        steps     = 0
        max_steps = 1000

        while not done and steps < max_steps:
            action, _, _ = ppo.select_action(state)
            state, _, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            if steps % 2 == 0:  # Save every other frame
                frames.append(env.render())
            steps += 1

        video_filename = f"ppo_hopper_trial_{trial_num + 1}.mp4"
        imageio.mimsave(video_filename, frames, fps=30)
        print(f"Video saved as '{video_filename}'")
    except Exception as e:
        print(f"Warning: Could not save video for trial {trial_num + 1}: {e}")

    env.close()
    return trial_rewards, ppo

def plot_rewards_with_trials(all_trial_rewards, window=100):
    """
    Plots training progress across all trials with confidence intervals.
    
    Args:
    - all_trial_rewards : List of rewards from each trial
    - window           : Window size for reward smoothing
    """
    plt.figure(figsize=(10, 6))
    rewards_array = np.array(all_trial_rewards)
    mean_rewards  = np.mean(rewards_array, axis=0)
    std_rewards   = np.std(rewards_array, axis=0)

    # Compute smoothed means using moving average
    smoothed_means = []
    for i in range(len(mean_rewards)):
        if i < window:
            smoothed_means.append(np.mean(mean_rewards[:i+1]))
        else:
            smoothed_means.append(np.mean(mean_rewards[i-window+1:i+1]))
    smoothed_means = np.array(smoothed_means)

    # Plot individual trials
    for trial_rewards in all_trial_rewards:
        plt.plot(trial_rewards, alpha=0.1, color='gray')

    # Plot mean with confidence interval
    episodes = range(len(mean_rewards))
    plt.plot(episodes, smoothed_means, linewidth=2, color='blue', label='Mean Reward')
    plt.fill_between(episodes, 
                     smoothed_means - std_rewards, 
                     smoothed_means + std_rewards, 
                     color='blue', alpha=0.2, label='Standard Deviation')

    plt.xlabel('Episode')
    plt.ylabel('Average Return')
    plt.title('PPO Training Progress - Hopper-v4')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig('ppo_training_progress.png', dpi=300, bbox_inches='tight')
    plt.close()

def main():
    num_trials        = 5
    all_trial_rewards = []
    env_name          = "Hopper-v4"
    for trial in range(num_trials):
        trial_rewards, _ = run_single_trial(trial, env_name)
        all_trial_rewards.append(trial_rewards)
        np.save(f'trial_rewards_{trial}.npy', trial_rewards)

    plot_rewards_with_trials(all_trial_rewards)
    print("\nTraining progress plot saved as 'ppo_training_progress.png'")

if __name__ == "__main__":
    main()



Starting Trial 1/5 with environment Hopper-v4


Trial 1:   0%|          | 1/1000 [00:02<41:46,  2.51s/it]


Trial 1, Episode 0, Average Reward: 1368.45


Trial 1:   1%|          | 11/1000 [00:23<35:14,  2.14s/it]


Trial 1, Episode 10, Average Reward: 2785.30


...


Trial 5, Episode 980, Average Reward: 6896.43


Trial 5:  99%|█████████▉| 991/1000 [31:17<00:16,  1.84s/it]


Trial 5, Episode 990, Average Reward: 6883.64


Trial 5: 100%|██████████| 1000/1000 [31:36<00:00,  1.90s/it]


Video saved as 'ppo_hopper_trial_5.mp4'

Training progress plot saved as 'ppo_training_progress.png'
